In [3]:
import os
import json
import faiss
import numpy as np
from openai import OpenAI
import traceback
import openai  # 추가
from dotenv import load_dotenv
import os
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# .env 파일 로드
load_dotenv('/upstage-ai-advanced-ir7/.env')

# API_KEY 값을 가져옴
upstage_api_key = os.getenv('UPSTAGE_API_KEY')
os.environ["OPENAI_API_KEY"] = upstage_api_key

# Upstage API 클라이언트 설정
client = OpenAI(
    api_key= upstage_api_key,
    base_url="https://api.upstage.ai/v1/solar"
)

client_gpt = OpenAI()

In [12]:
with open("/upstage-ai-advanced-ir7/data/eval.jsonl", "r") as f:
    eval_doc_mapping = [json.loads(line) for line in f]
    

doc_mapping = {}
with open("/upstage-ai-advanced-ir7/data/documents.jsonl", "r") as f:
    for line in f:
        doc = json.loads(line)
        doc_mapping[doc['docid']] = doc 

index = faiss.read_index("knn_index_cosine.faiss")

# gpu_index = faiss.index_cpu_to_gpu(res, 0, index)

with open("chunk_mappings.json", "r") as f:
    chunk_doc_mapping = json.load(f)
    
model_path = 'Dongjin-kr/ko-reranker'

def exp_normalize(x):
    b = x.max()
    y = np.exp(x - b)
    return y / y.sum()
    
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.to('cuda')
model.eval()

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, ou

순서
1. Query Transformation
2. Non-Science Question Detector

#### 1. Query Transformer

In [3]:
def query_transformer(messages):
    """
    LLM을 사용하여 사용자의 여러 메시지를 기반으로 검색에 적합한 단일 쿼리 생성.
    """
    # 시스템 메시지로 LLM에게 과제 부여 (한국어로)
    system_message = {
        "role": "system",
        "content": """
        당신은 사용자의 여러 대화 메시지를 하나의 검색 쿼리로 변환하는 전문가입니다. 
        사용자의 대화 중 중요한 정보만을 추출하여 간결하고 명확한 검색용 쿼리로 변환하세요. 
        대화 형식이 아닌, 연구나 조사를 위한 검색어처럼 정확한 질문을 생성하세요.
        """
    }

    # 사용자 메시지 준비
    dialogue_messages = [{"role": msg['role'], "content": msg['content']} for msg in messages]

    # LLM 호출을 위한 메시지 배열 생성
    full_message = [system_message] + dialogue_messages

    # OpenAI API 호출하여 적절한 검색 쿼리 생성
    result = client_gpt.chat.completions.create(
        model="gpt-4o",  # LLM 모델 지정
        messages=full_message,
        temperature=0
    )

    # LLM이 생성한 쿼리 반환 (ChatCompletionMessage 형식에서 content에 직접 접근)
    transformed_query = result.choices[0].message.content
    print(f"변환된 쿼리: {transformed_query}")
    return transformed_query

In [9]:
messages = [
    {"role": "user", "content": "기억 상실증 걸리면 너무 무섭겠다."},
    {"role": "assistant", "content": "네 맞습니다."},
    {"role": "user", "content": "원인이 뭘까."}
]

In [11]:
transformed_query = query_transformer(messages)

변환된 쿼리: 기억 상실증의 원인


In [12]:
transformed_query

'기억 상실증의 원인'

#### 2. Non-Science Query Detector

In [5]:
def science_query_detector(message):
    """
    LLM을 사용하여 단일 사용자의 질문을 검색에 적합한 쿼리로 변환.
    질문이 과학과 관련되어 않으면 '과학 관련 질문이 아닙니다'라고 답변.
    """
    # 시스템 메시지로 LLM에게 과제 부여 (한국어로)
    system_message = {
        "role": "system",
        "content": """
            당신은 과학적 또는 학문적 논의와 관련된 질문에 답변하는 전문가입니다.
            질문이 전혀 과학적이지 않은 경우에만 '과학 관련 질문이 아닙니다.'라고 답변하세요.
            질문이 과학과 조금이라도 아주 조금이라도 관련이 있다면, 예를 들어 음식, 여가 등 매우 기초적인 것이라도 그 질문을 구체적이고 명확한 검색 쿼리로 변환하세요.
            과학적 논의는 자연과학, 사회과학, 기술, 심리학, 역사적 연구 등 다양한 학문 분야를 포함할 수 있습니다.
            단, 변환된 쿼리를 명확히 출력하되, 변환 과정을 설명하거나 예시로 출력하지 말고, **변환된 쿼리만 출력하세요.**
            
            예를 들어:
            - '복잡한 데이터 구조 설계 방법을 알려줘.' -> '복잡한 데이터 구조 설계 방법'
            - '인간의 감정이 사회적 관계에 미치는 영향' -> '인간의 감정이 사회적 관계에 미치는 영향'
            변환된 쿼리만 출력하세요.
        """
    }

    # 사용자 질문 준비
    user_message = {"role": message[0]['role'], "content": message[0]['content']}

    # LLM 호출을 위한 전체 메시지 배열 생성
    full_message = [system_message, user_message]

    # OpenAI API 호출하여 적절한 검색 쿼리 생성
    result = client_gpt.chat.completions.create(
        model="gpt-4o",  # LLM 모델 지정
        messages=full_message,
        temperature=0
    )

    # LLM이 생성한 응답을 반환
    llm_response = result.choices[0].message.content

    # 과학 관련 질문 여부 판단 및 처리
    if "과학 관련 질문이 아닙니다" in llm_response:
        print("과학 관련 질문이 아닙니다.")
        return "과학 관련 질문이 아닙니다."
    else:
        # 과학 관련 질문일 경우 변환된 쿼리 반환
        refined_query = llm_response.strip()
        print(f"변환된 쿼리: {refined_query}")
        return refined_query

In [18]:
message = [
    {"role": "user", "content": "과학공부 하기 싫어"}
]

In [19]:
science_query_detector(message)

과학 관련 질문이 아닙니다.


'과학 관련 질문이 아닙니다.'

#### 3. Retrival KNN + Reranker

In [13]:
def normalize(embeddings):
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return (embeddings / norms).astype(np.float32)  # float32로 변환

def retrieval(query):
    query_result = client.embeddings.create(
        model="solar-embedding-1-large-query",
        input=query
    ).data[0].embedding

    query_embedding = np.array(query_result).reshape(1, -1)

    # 쿼리 임베딩 정규화
    normalized_query = normalize(query_embedding)

    # FAISS로 50개의 유사한 chunk 검색
    k = 200
    distances, indices = index.search(normalized_query, k)

    retrieved_chunks = []
    retrieved_doc_ids = []
    for idx in indices[0]:
        chunk_info = chunk_doc_mapping[idx]
        content = chunk_info['content']
        doc_id = chunk_info['doc_id']
        retrieved_chunks.append((query, content))
        retrieved_doc_ids.append(doc_id)
        # print(f"검색된 Chunk ID: {chunk_info['chunk_id']}")
        # print(f"연관된 문서 ID: {doc_id}")
        # print(f"Chunk 내용: {content}")
        # print("-" * 50)
        
    with torch.no_grad():
        inputs = tokenizer(retrieved_chunks, padding=True, truncation=True, return_tensors='pt', max_length=512).to('cuda')
        scores = model(**inputs, return_dict=True).logits.view(-1, ).float()

    # 상위 5개의 문서 선택 (내림차순 정렬)
    top_5_indices = torch.argsort(scores, descending=True)[:10]

    # 상위 5개의 문서 출력
    final_doc_ids = []
    print("상위 5개의 문서:")
    for i in top_5_indices:
        doc_index = retrieved_chunks[i][0]
        doc_text = retrieved_chunks[i][1]
        doc_id = retrieved_doc_ids[i]
        final_doc_ids.append(doc_id)
        # print(f"문서 인덱스: {doc_index}")
        # print(f"연관된 문서 ID: {doc_id}")
        # print(f"문서 내용: {doc_text}")
        
        # print(f"유사도 점수: {scores[i].item()}")
        # print("-" * 50)
    return final_doc_ids

In [871]:
retreived_doc_ids = retrieval('금성에서 달의 관측 모습')

상위 5개의 문서:


In [605]:
retreived_doc_ids

[('82b095fd-2fb6-48ae-8476-2f457f8b6650', -2.3021411895751953),
 ('4344f76e-1747-4bc9-8d02-c26db29151f4', -6.517877101898193),
 ('c8fd4323-9af9-4a0d-ab53-e563c71f9795', -6.6319732666015625),
 ('ac826442-684c-4ea4-b602-e356fd63bde8', -6.711040019989014),
 ('c1edd0df-b9f4-4dc2-90f4-38708cddddba', -6.8428263664245605)]

In [88]:
d = doc_mapping['25de4ffd-cee4-4f27-907e-fd6b802c6ede']

In [96]:
y = {"docid": "82b095fd-2fb6-48ae-8476-2f457f8b6650", "src": "ko_ai2_arc__ARC_Challenge__train", "content": "인간 남성과 여성의 생식세포는 같은 수의 염색체를 가지고 있습니다. 남성은 23쌍의 염색체를 가지고 있으며, 여성도 23쌍의 염색체를 가지고 있습니다. 따라서, 일반적인 남성과 여성의 생식세포가 결합하여 자손을 생산할 때, 염색체의 수는 두 배가 됩니다. 이는 염색체의 유전 정보를 조합하여 새로운 개체를 형성하는 과정입니다. 염색체는 DNA를 포함하고 있으며, DNA는 유전 정보를 담고 있는 분자입니다. 따라서, 염색체의 수가 두 배가 되면, 자손은 부모의 유전 정보를 조합하여 새로운 개체를 형성하게 됩니다."}

In [95]:
d

{'docid': '25de4ffd-cee4-4f27-907e-fd6b802c6ede',
 'src': 'ko_mmlu__high_school_biology__test',
 'content': '기억 상실은 대뇌의 기능 장애로 인해 발생할 가능성이 가장 높습니다. 대뇌는 인간의 중추신경계에서 가장 중요한 역할을 담당하며, 인지, 기억, 감정 등 다양한 기능을 조절합니다. 따라서 대뇌의 어느 부분에서 문제가 발생하면 기억 상실이나 기타 인지 기능의 장애가 발생할 수 있습니다. 대뇌는 뇌의 가장 큰 부분으로, 뇌의 표면에 위치한 뇌피질과 그 아래에 있는 심층뇌로 구성되어 있습니다. 이러한 대뇌의 구조와 기능이 원활하게 작동하지 않으면 기억 상실이 발생할 수 있습니다. 따라서 대뇌는 기억 상실과 관련된 가장 중요한 부분으로 간주됩니다.'}

#### 4. Document Relevance Check


In [8]:
def document_relevance_checker(document):
    """
    LLM을 사용하여 문서에서 논리적 오류, 상식적 타당성, 과학적 평가 및 수학적 평가를 진행합니다.
    네 가지 조건이 모두 충족되지 않을 경우 'False'를 반환하고, 그렇지 않으면 'True'를 반환합니다.
    """
    system_message = {
        "role": "system",
        "content": """
        당신은 과학적 문서를 평가하는 전문가입니다.

        **1. 논리적 오류 평가**:
            - 문서의 설명에서 과학적 원리와 모순되거나 자명한 논리적 오류가 있는지 확인하세요.
            - 예: '해머로 철벽을 치는 것이 열을 발생시키기 위한 목적이다'라는 설명은 과학적 원리와 어긋납니다.

        **2. 상식적 타당성 평가**:
            - 설명이 과학적으로 맞다고 해도 현실적인지 상식적으로 타당한지 판단하세요.
            - 예: '가스레인지를 이용해 집안을 따뜻하게 한다'는 설명은 상식적이지 않을 수 있습니다.

        **3. 과학적 평가**:
            - 문서의 설명이 학계에서 널리 받아들여지는 과학적 사실과 일치하는지 평가하세요.
            - 예: 생식세포의 염색체 수가 23쌍(46개)라는 사실은 맞지만, '염색체의 수가 두 배가 된다'는 해석은 잘못된 것입니다.

        **4. 수학적 평가**:
            - 문서에 포함된 수학적 표현이 논리적이고 타당한지 확인하세요.
            - 예: '매년 13개월의 월급을 지급합니다'는 수학적으로 잘못된 표현입니다.

        **결과**:
            - 네 가지 평가 기준 중 두개이상 충족되면 "True"로 답변하세요.
            - 네 가지 중 3가지 기준이상 틀렸을 경우 "False"로 답변하세요.
        """
    }

    # 사용자 문서 입력 준비
    document_message = {"role": "user", "content": f"문서 출처: {document['src']}\n내용: {document['content']}"}

    # LLM 호출을 위한 전체 메시지 배열 생성
    full_message = [system_message, document_message]
    # OpenAI API 호출하여 적절한 질문 생성
    result = client_gpt.chat.completions.create(
        model="gpt-4o",  # 사용할 모델 지정
        messages=full_message,
        temperature=0
    )
    
    
    print(document['content'])
    # LLM이 생성한 응답을 가져오기
    llm_response = result.choices[0].message.content

    if llm_response == 'False':
        print("Logical Fallacy")
    else:
        print("Good Doc")

    return llm_response
    # 논리적 오류와 상식적 타당성을 반환하거나, 논리적인 질문 5개를 반환
    # if "논리적 오류가 있으며, 상식적이지 않습니다" in llm_response:
    #     print("문서가 논리적 오류가 있으며, 상식적이지 않습니다.")
    #     return "이 문서는 논리적 오류가 있으며, 상식적이지 않습니다."
    # else:
    #     # 논리적인 질문 5개를 반환
    #     questions = llm_response.strip().split("\n")
    #     print(f"생성된 질문들: {questions}")
    #     return questions

In [9]:
def document_relevance_checker(document, query):
    """
    LLM을 사용하여 문서에서 논리적 오류, 상식적 타당성, 과학적 평가, 수학적 평가 및 쿼리 연관도를 바탕으로 점수를 계산합니다.
    각 항목에서 0~200점 사이의 점수를 부여하여 최종 점수를 반환합니다.
    """

    # 프롬프트에 평가 기준과 점수화 요구를 설정
    system_message = {
        "role": "system",
        "content": """
        당신은 문서의 적합성을 평가하는 전문가입니다.
        각 항목에 대해 0~200점 사이의 점수를 부여하세요:

        **1. 논리적 오류 평가**:
            - 문서에서 과학적 원리와 모순되거나 논리적 오류가 있으면 감점하세요.
            - 예: '해머로 철벽을 치는 것이 열을 발생시키기 위한 목적이다'라는 설명은 논리적 오류입니다.
            - 논리적으로 완벽할 경우 200점, 일부 모순이 있을 경우 100~150점, 명백한 오류가 있을 경우 0~50점을 부여하세요.

        **2. 상식적 타당성 평가**:
            - 설명이 상식적으로 타당한지 평가하세요.
            - 예: '가스레인지를 이용해 집안을 따뜻하게 한다'는 설명은 상식적이지 않으며, 감점 대상입니다.
            - 상식적으로 완벽할 경우 200점, 다소 현실성 부족한 경우 100~150점, 상식적으로 어긋날 경우 0~50점을 부여하세요.

        **3. 과학적 평가**:
            - 문서의 설명이 학계에서 널리 받아들여지는 과학적 사실과 일치하는지 평가하세요.
            - 예: 생식세포의 염색체 수가 23쌍(46개)라는 사실은 맞지만, '염색체의 수가 두 배가 된다'는 해석은 잘못된 것입니다.
            - 과학적으로 타당할 경우 200점, 일부 오해의 소지가 있을 경우 100~150점, 명백한 오류일 경우 0~50점을 부여하세요.

        **4. 수학적 평가**:
            - 문서에서 수학적 오류나 논리적 모순이 있는지 평가하세요.
            - 예: '매년 13개월의 월급을 지급합니다'는 수학적으로 잘못된 표현입니다.
            - 수학적으로 타당할 경우 200점, 일부 계산 오류가 있을 경우 100~150점, 명백한 오류일 경우 0~50점을 부여하세요.

        **5. 쿼리 연관도 평가**:
            - 질문(query)과 문서의 내용이 얼마나 연관되어 있는지 평가하세요.
            - 문서가 질문과 밀접하게 연관되어 있을 경우 200점, 일부 관련이 있을 경우 100~150점, 관련성이 거의 없을 경우 0~50점을 부여하세요.

        **결과**:
            - 각 항목의 점수를 0~200점 사이로 부여하고, 최종 점수를 계산하세요.
            - 최종 점수를 정수로 숫자로 답하시오.
            - 숫자만 생성하세요.
            - 최종 점수 외엔 아무것도 생성하지 마세요
        """
    }

    # 사용자 쿼리 및 문서 메시지
    query_message = {"role": "user", "content": f"질문: {query}"}
    document_message = {"role": "user", "content": f"문서 내용: {document['content']}"}

    # LLM 호출을 위한 전체 메시지 배열 생성
    full_message = [system_message, query_message, document_message]

    # OpenAI API를 사용하여 LLM 호출
    result = client_gpt.chat.completions.create(
        model="gpt-4o",  # 사용할 모델 지정
        messages=full_message,
        temperature=0
    )
    print(document['content'])
    # LLM이 생성한 점수를 받아오기 (최종 점수는 1000점 만점)
    llm_response = result.choices[0].message.content.strip()
    
    import re
    scores = re.findall(r'\d+', llm_response)  # 모든 숫자 추출
    if scores:
        final_score = sum(map(int, scores)) // len(scores)  # 최종 점수 계산
    else:
        final_score = 0

    # 점수 출력
    print(f"문서 평가 최종 점수: {int(final_score)}/200")

    return final_score

In [645]:
document_relevance_checker(z)

TypeError: document_relevance_checker() missing 1 required positional argument: 'query'

In [468]:
z = {"docid": "41ca41ac-66e3-4a6b-a604-87bf8b3a8d4d", "src": "ko_ai2_arc__ARC_Challenge__test", "content": "한 건설 작업자가 해머로 단단한 철벽을 치려 했지만 벽은 움직이지 않았습니다. 작업자가 사용한 에너지의 일부는 열로 변환되었습니다. 철벽은 고강도 강재로 만들어져 있어서 해머로 강력하게 치더라도 움직이지 않는 것이 정상입니다. 철벽은 건물의 구조를 지탱하고 보호하기 위해 설계되었으며, 그 내부에는 철과 다른 금속들이 견고하게 결합되어 있습니다. 따라서 해머로 치는 작업은 벽을 움직이게 하는 것보다는 에너지를 전달하여 열로 변환시키는 역할을 합니다. 이러한 열 변환은 작업자가 사용한 에너지를 벽에 흡수시키고, 벽의 온도를 조절하는 역할을 합니다. 따라서 작업자가 해머로 철벽을 치는 것은 벽을 움직이게 하는 것보다는 벽과 상호작용하여 열을 발생시키는 과정입니다."}


In [688]:
retreived_doc_ids

[('41ca41ac-66e3-4a6b-a604-87bf8b3a8d4d', -0.5539787411689758),
 ('8132c4ab-f1bd-4915-8196-a13fc3df1f05', -3.493497610092163),
 ('6211fd17-f7cf-4680-82be-365da49b211e', -4.1826372146606445),
 ('a395b38f-af11-409e-9c17-89a21f4a9521', -4.888504981994629),
 ('96459005-ecc6-4db9-914a-14043f280a08', -5.461760997772217)]

인간의 DNA 결합 과정
기름과 물의 혼합 가능성
금성에서 달의 관측 모습

In [877]:
final_ids = []
final_ids_2 = []
for doc_id in retreived_doc_ids:
    current_doc = doc_mapping[doc_id[0]]
    checked = document_relevance_checker(current_doc, '금성에서 달의 관측 모습')
    final_ids.append((doc_id[0], checked))
    # if checked == 'True':
    #     final_ids.append(doc_id[0])
    # else:
    #     final_ids_2.append(doc_id[0])

당신은 금성에 살고 있고, 당신의 망원경이 금성의 두꺼운 구름을 볼 수 있다고 가정한다면, 지구의 달을 관찰할 수 있습니다. 지구의 달은 월식과 만월을 거치며 다양한 단계를 보여줍니다. 여기서는 월식이 아닌 단계를 중심으로 설명하겠습니다.

첫 번째로, 상현 단계를 볼 수 있습니다. 이 단계에서는 달이 점점 커지면서 반달 모양을 보여줍니다. 달의 오른쪽 절반은 밝고, 왼쪽 절반은 어둡습니다. 이는 달이 점점 지구와 태양 사이의 각도가 커지면서 발생하는 현상입니다.

두 번째로, 망 단계를 볼 수 있습니다. 이 단계에서는 달이 지구와 태양 사이에 정확히 위치하여 완전한 원형을 보여줍니다. 이는 달이 지구와 태양 사이의 각도가 180도가 되는 순간입니다. 이때 달은 가장 밝게 보입니다.

세 번째로, 하현 단계를 볼 수 있습니다. 이 단계에서는 달이 점점 작아지면서 반달 모양을 보여줍니다. 달의 왼쪽 절반은 밝고, 오른쪽 절반은 어둡습니다. 이는 달이 점점 지구와 태양 사이의 각도가 작아지면서 발생하는 현상입니다.

따라서, 당신이 금성에 살고 있다면 상현, 망, 하현 단계의 지구의 달을 관찰할 수 있을 것입니다.
문서 평가 최종 점수: 0/1000
달이 정오에 지면 달의 위상은 상현달입니다. 상현달은 달의 위상 중 하나로, 달이 지구와 태양 사이에 위치하여 태양의 빛을 받아 반사되는 달의 모습을 말합니다. 이 때 달은 지구에서 볼 때 반달 모양으로 보이며, 달의 오른쪽 절반은 밝고 왼쪽 절반은 어둡게 보입니다. 이러한 상현달은 달의 위상 중 가장 두드러진 모습으로 알려져 있습니다. 달의 위상은 달이 지구 주위를 공전하면서 변하는데, 이는 태양과 달의 상대적인 위치에 따라 결정됩니다. 따라서 달의 위상은 매일 변하며, 상현달은 그 중 하나입니다. 상현달은 우리가 보는 달의 모습을 통해 달의 움직임과 태양계의 운동을 이해하는 데 도움을 줍니다.
문서 평가 최종 점수: 0/1000
금성이 다른 행성들보다 더 밝게 보이는 이유는 지구 쪽으로 가장 많은 햇빛을 반사하기 

In [878]:
sorted_ids = sorted(final_ids, key=lambda x: x[1], reverse=True)


In [879]:
sorted_ids

[('35c5dcc7-4720-4318-901e-770105ae63fd', 0),
 ('553989d9-ee23-4203-b244-a941b6fa8d99', 0),
 ('464ace62-ddf2-423d-a5d7-2f17e6785c8e', 0),
 ('8a78364e-63bf-4915-b718-fdc461bc62c9', 0),
 ('59a8259f-4a39-4ab6-ad3f-2e4161ad458d', 0)]

In [880]:
final_ids

[('35c5dcc7-4720-4318-901e-770105ae63fd', 0),
 ('553989d9-ee23-4203-b244-a941b6fa8d99', 0),
 ('464ace62-ddf2-423d-a5d7-2f17e6785c8e', 0),
 ('8a78364e-63bf-4915-b718-fdc461bc62c9', 0),
 ('59a8259f-4a39-4ab6-ad3f-2e4161ad458d', 0)]

#### 5. Top Agent

In [885]:
data = [
    ('a', 100),
    ('b', 50),
    ('c', 150),
    ('d', 30),
    ('e', 200),
    ('f', 75)
]

data[1:]

[('b', 50), ('c', 150), ('d', 30), ('e', 200), ('f', 75)]

In [886]:
data = [
    ('a', 100),
    ('b', 50),
    ('c', 150),
    ('d', 30),
    ('e', 200),
    ('f', 75)
]

# 슬라이스 범위 지정 (예: 인덱스 1부터 4까지 정렬)
start_index = 1
end_index = 4

# 슬라이스 부분을 정렬 (두 번째 요소 기준으로 오름차순 정렬)
sorted_slice = sorted(data[start_index:], key=lambda x: x[1])

# 정렬된 슬라이스를 원본 리스트의 해당 위치에 삽입
data[start_index:] = sorted_slice

In [887]:
data

[('a', 100), ('d', 30), ('b', 50), ('f', 75), ('c', 150), ('e', 200)]

In [10]:
def top_agent(queries):
    if len(queries) > 1:
        transformed_query = query_transformer(queries)
        query = [
            {"role": "user", "content": transformed_query}
        ]
        checked_query = science_query_detector(query)
        
        if checked_query == '과학 관련 질문이 아닙니다.':
            return False
    else:
        checked_query = science_query_detector(queries)
        if checked_query == '과학 관련 질문이 아닙니다.':
            return False

    retrieved_doc = retrieval(checked_query)
    
    
    final_result = [checked_query]
    for doc_id in retrieved_doc:
        current_doc = doc_mapping[doc_id]
        checked = document_relevance_checker(current_doc, checked_query)
        final_result.append((doc_id, checked))
        
    sorted_slice = sorted(final_result[1:], key=lambda x: x[1], reverse=True)
    final_result[1:] = sorted_slice
    return final_result

In [910]:
top_agent_result = top_agent(messages)

변환된 쿼리: 기억 상실증의 원인
변환된 쿼리: 기억 상실증의 원인
상위 5개의 문서:
기억 상실은 대뇌의 기능 장애로 인해 발생할 가능성이 가장 높습니다. 대뇌는 인간의 중추신경계에서 가장 중요한 역할을 담당하며, 인지, 기억, 감정 등 다양한 기능을 조절합니다. 따라서 대뇌의 어느 부분에서 문제가 발생하면 기억 상실이나 기타 인지 기능의 장애가 발생할 수 있습니다. 대뇌는 뇌의 가장 큰 부분으로, 뇌의 표면에 위치한 뇌피질과 그 아래에 있는 심층뇌로 구성되어 있습니다. 이러한 대뇌의 구조와 기능이 원활하게 작동하지 않으면 기억 상실이 발생할 수 있습니다. 따라서 대뇌는 기억 상실과 관련된 가장 중요한 부분으로 간주됩니다.
문서 평가 최종 점수: 200/200
치매는 뇌손상이 되돌릴 수 없고 증가하고 있는 질환입니다. 이 질환은 뇌 기능의 저하와 기억력 손실을 초래합니다. 치매는 주로 노인들에게서 발생하지만, 어떤 경우에는 어린이나 중년층에서도 발생할 수 있습니다. 치매는 점진적으로 증상이 악화되며, 일상 생활에 심각한 영향을 미칩니다. 이 질환은 현재로서는 완전히 치료할 수 없지만, 조기 발견과 관리를 통해 증상의 진행을 늦출 수 있습니다. 치매에 대한 연구는 계속 진행되고 있으며, 예방과 치료에 대한 새로운 방법들이 개발되고 있습니다. 치매는 많은 사람들에게 영향을 미치는 질환으로, 사회적인 문제로도 대두되고 있습니다. 따라서 치매에 대한 인식과 이해를 높이는 것이 중요합니다. 치매는 현재로서는 완전히 치료할 수 없지만, 연구와 노력을 통해 이 질환을 극복할 수 있는 방법들이 발전될 것을 기대합니다.
문서 평가 최종 점수: 50/200
일화적 기억은 나이가 들수록 가장 큰 결핍을 보이는 기억 유형입니다. 이는 일상적인 사건이나 경험에 대한 기억을 의미합니다. 나이가 들면서 기억력이 저하되고, 일화적인 기억을 잘 기억하지 못하는 경우가 많습니다. 이는 노화로 인해 뇌의 기능이 감소하고, 정보 처리 속도가 느려지는 것과 관련이 있습니다. 따라서, 일화적 기억

In [908]:
message = [
    {"role": "user", "content": "요새 너무 힘들다."}
]

In [911]:
top_agent_result

['기억 상실증의 원인',
 ('25de4ffd-cee4-4f27-907e-fd6b802c6ede', 200),
 ('df495f22-6315-42a8-9553-b43ab707b683', 100),
 ('2566f872-32e5-4def-8ed9-3341740e81e8', 100),
 ('0c2d5267-638a-4954-8974-55d28a3fe600', 100),
 ('ad56325d-400f-416c-b5de-2053eedecac4', 50)]

In [903]:
top_agent_result

['기억 상실증 원인',
 ('25de4ffd-cee4-4f27-907e-fd6b802c6ede', 200),
 ('2566f872-32e5-4def-8ed9-3341740e81e8', 150),
 ('df495f22-6315-42a8-9553-b43ab707b683', 150),
 ('ad56325d-400f-416c-b5de-2053eedecac4', 50),
 ('0c2d5267-638a-4954-8974-55d28a3fe600', 50)]

In [218]:
message

[{'role': 'user', 'content': '글루텐이 에너지 흡수에 미치는 영향은?'}]

In [597]:
top_agent_result = ['query', ['doc1', 0.9], ['doc2', 0.8]]
print(top_agent_result[1:4])

[['doc1', 0.9], ['doc2', 0.8]]


In [221]:
eval_doc_mapping[0]

{'eval_id': 78,
 'msg': [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]}

In [912]:
with open('/upstage-ai-advanced-ir7/data/eval.jsonl') as f, open('/upstage-ai-advanced-ir7/output_10_07.csv', "w") as of:
    for line in f:
        eval_doc = json.loads(line)
        top_agent_result = top_agent(eval_doc['msg'])
        if top_agent_result ==  False:
            output = {"eval_id": eval_doc['eval_id'], "standalone_query": '', "topk": [], "answer": eval_doc['msg'][0]['content'], "references": [] }
            print(eval_doc['msg'][0]['content'])

        else:   
            output = {"eval_id": eval_doc['eval_id'], "standalone_query": top_agent_result[0], "topk": [], "answer": doc_mapping[top_agent_result[1][0]]['content'], "references": [] }
            for doc in top_agent_result[1:4]:
                output['topk'].append(doc[0])
                # doc_mapping[doc[0]]['content']
                output['references'].append({"score" : doc[1], "content" : doc_mapping[doc[0]]['content']})
            print('*' * 100)
        of.write(f'{json.dumps(output, ensure_ascii=False)}\n')

변환된 쿼리: 나무의 분류 방법 연구
상위 5개의 문서:
한 학생이 다양한 종류의 나무를 조사하고 있습니다. 이 학생은 성장 속도, 온도 범위, 크기가 비슷한 두 나무를 발견했습니다. 그러나 이 두 나무의 잎과 꽃은 서로 다릅니다. 이러한 특징을 고려하면, 이 나무들은 대체로 같은 속에 속해 있을 것으로 추측됩니다. 같은 속에 속한 나무들은 종류별로 유사한 특징을 가지고 있으며, 이는 생물 분류학에서 중요한 기준 중 하나입니다. 따라서 이 학생의 조사 결과는 나무의 분류와 관련된 중요한 정보를 제공할 수 있습니다. 이러한 조사는 나무의 성장과 생태에 대한 이해를 높이는 데 도움이 될 것입니다.
문서 평가 최종 점수: 190/200
나무는 재생 가능한 에너지원으로 간주됩니다. 이는 나무가 석탄이 형성되는 것보다 빠르게 자라기 때문입니다. 나무는 태양 에너지를 흡수하여 광합성을 통해 자라납니다. 이 과정에서 나무는 이산화탄소를 흡수하고 산소를 방출합니다. 이러한 특성으로 인해 나무는 지속적으로 재생되는 에너지원으로 사용될 수 있습니다. 반면, 석탄은 수백만 년 동안 지속적인 압력과 열에 노출되어 형성됩니다. 석탄은 화석 연료로 분류되며, 한 번 사용되면 재생되지 않습니다. 따라서 나무는 석탄보다 더욱 지속 가능한 에너지원으로 간주됩니다.
문서 평가 최종 점수: 0/200
찰스 다윈은 젊은 생물학자로서 남미의 온대 지역 식물과 유럽의 온대 지역 식물이 유사할 것으로 예상했습니다. 그러나 그는 놀랍게도 남미의 열대 지역 식물과 더 유사하다는 것을 발견했습니다. 이 관찰은 생물지리학과 가장 적절하게 연관되어 있습니다. 생물지리학은 생물의 분포와 분류를 연구하는 학문으로, 지리적인 요인이 생물의 진화와 다양성에 어떤 영향을 미치는지를 연구합니다. 다윈의 관찰은 지리적인 요인이 식물의 진화에 영향을 미치는 것을 보여주었으며, 이는 생물지리학의 중요성을 강조합니다.
문서 평가 최종 점수: 50/200
축축한 습지에서 자라고 있는 식물은 비종자 유관속식물로 분류됩니다. 비종자

In [14]:
with open('/upstage-ai-advanced-ir7/data/eval.jsonl') as f, open('/upstage-ai-advanced-ir7/output_10_08.csv', "w") as of:
    for line in f:
        eval_doc = json.loads(line)
        top_agent_result = top_agent(eval_doc['msg'])
        if top_agent_result ==  False:
            output = {"eval_id": eval_doc['eval_id'], "standalone_query": '', "topk": [], "answer": eval_doc['msg'][0]['content'], "references": [] }
            print(eval_doc['msg'][0]['content'])

        else:   
            output = {"eval_id": eval_doc['eval_id'], "standalone_query": top_agent_result[0], "topk": [], "answer": doc_mapping[top_agent_result[1][0]]['content'], "references": [] }
            for doc in top_agent_result[1:4]:
                output['topk'].append(doc[0])
                # doc_mapping[doc[0]]['content']
                output['references'].append({"score" : doc[1], "content" : doc_mapping[doc[0]]['content']})
            print('*' * 100)
        of.write(f'{json.dumps(output, ensure_ascii=False)}\n')

변환된 쿼리: '나무의 분류 방법 조사'
상위 5개의 문서:
한 학생이 다양한 종류의 나무를 조사하고 있습니다. 이 학생은 성장 속도, 온도 범위, 크기가 비슷한 두 나무를 발견했습니다. 그러나 이 두 나무의 잎과 꽃은 서로 다릅니다. 이러한 특징을 고려하면, 이 나무들은 대체로 같은 속에 속해 있을 것으로 추측됩니다. 같은 속에 속한 나무들은 종류별로 유사한 특징을 가지고 있으며, 이는 생물 분류학에서 중요한 기준 중 하나입니다. 따라서 이 학생의 조사 결과는 나무의 분류와 관련된 중요한 정보를 제공할 수 있습니다. 이러한 조사는 나무의 성장과 생태에 대한 이해를 높이는 데 도움이 될 것입니다.
문서 평가 최종 점수: 200/200
킨제이 척도는 성적 지향성을 측정하는 도구로, 사람들을 다양한 등급으로 분류합니다. 4등급을 가진 사람들은 상당한 이성애 경험이 있는 동성애자로 분류됩니다. 이 등급은 이성애와 동성애 경험을 모두 가지고 있으며, 이성애에 대한 강한 성적 지향성을 가지고 있습니다. 이 등급에 속하는 사람들은 동성애 경험이 있지만, 이성애에 대한 성적인 끌림이 여전히 존재합니다. 이러한 사람들은 성적 지향성이 다양하며, 이성애와 동성애 모두에 대한 성적인 욕구를 가지고 있을 수 있습니다. 킨제이 척도는 성적 지향성을 이해하고 분류하는 데 도움을 주는 중요한 도구입니다.
문서 평가 최종 점수: 0/200
나무는 재생 가능한 에너지원으로 간주됩니다. 이는 나무가 석탄이 형성되는 것보다 빠르게 자라기 때문입니다. 나무는 태양 에너지를 흡수하여 광합성을 통해 자라납니다. 이 과정에서 나무는 이산화탄소를 흡수하고 산소를 방출합니다. 이러한 특성으로 인해 나무는 지속적으로 재생되는 에너지원으로 사용될 수 있습니다. 반면, 석탄은 수백만 년 동안 지속적인 압력과 열에 노출되어 형성됩니다. 석탄은 화석 연료로 분류되며, 한 번 사용되면 재생되지 않습니다. 따라서 나무는 석탄보다 더욱 지속 가능한 에너지원으로 간주됩니다.
문서 평가 최종 점수: 0/200
나무는 시

In [234]:
top_agent_result == False

True

In [226]:
output

{'eval_id': 78,
 'standalone_query': '나무의 분류 방법 연구',
 'topk': ['c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d',
  '8018337f-15cb-4341-b6fa-e311b4372df9',
  'd42ced41-7d0c-4346-bc0a-11454f5b6121'],
 'answer': '한 학생이 다양한 종류의 나무를 조사하고 있습니다. 이 학생은 성장 속도, 온도 범위, 크기가 비슷한 두 나무를 발견했습니다. 그러나 이 두 나무의 잎과 꽃은 서로 다릅니다. 이러한 특징을 고려하면, 이 나무들은 대체로 같은 속에 속해 있을 것으로 추측됩니다. 같은 속에 속한 나무들은 종류별로 유사한 특징을 가지고 있으며, 이는 생물 분류학에서 중요한 기준 중 하나입니다. 따라서 이 학생의 조사 결과는 나무의 분류와 관련된 중요한 정보를 제공할 수 있습니다. 이러한 조사는 나무의 성장과 생태에 대한 이해를 높이는 데 도움이 될 것입니다.',
 'references': [{'score': 2.703155040740967,
   'content': '한 학생이 다양한 종류의 나무를 조사하고 있습니다. 이 학생은 성장 속도, 온도 범위, 크기가 비슷한 두 나무를 발견했습니다. 그러나 이 두 나무의 잎과 꽃은 서로 다릅니다. 이러한 특징을 고려하면, 이 나무들은 대체로 같은 속에 속해 있을 것으로 추측됩니다. 같은 속에 속한 나무들은 종류별로 유사한 특징을 가지고 있으며, 이는 생물 분류학에서 중요한 기준 중 하나입니다. 따라서 이 학생의 조사 결과는 나무의 분류와 관련된 중요한 정보를 제공할 수 있습니다. 이러한 조사는 나무의 성장과 생태에 대한 이해를 높이는 데 도움이 될 것입니다.'},
  {'score': -3.1997716426849365,
   'content': '나무는 재생 가능한 에너지원으로 간주됩니다. 이는 나무가 석탄이 형성되는 것보다 빠르게 자라기 때문입니다. 나무는 태양 에너지를 흡수하여 광합

In [167]:
top_agent_result

['기억 상실증의 원인',
 ('25de4ffd-cee4-4f27-907e-fd6b802c6ede', 3.178738832473755),
 ('ad56325d-400f-416c-b5de-2053eedecac4', -1.7741055488586426),
 ('df495f22-6315-42a8-9553-b43ab707b683', -2.4628868103027344),
 ('2566f872-32e5-4def-8ed9-3341740e81e8', -2.5218491554260254),
 ('0c2d5267-638a-4954-8974-55d28a3fe600', -2.5466132164001465)]

In [182]:
for doc in top_agent_result[1:4]:
    print(doc)

('25de4ffd-cee4-4f27-907e-fd6b802c6ede', 3.178738832473755)
('ad56325d-400f-416c-b5de-2053eedecac4', -1.7741055488586426)
('df495f22-6315-42a8-9553-b43ab707b683', -2.4628868103027344)


In [195]:
doc_mapping[top_agent_result[1][0]]['content']

'기억 상실은 대뇌의 기능 장애로 인해 발생할 가능성이 가장 높습니다. 대뇌는 인간의 중추신경계에서 가장 중요한 역할을 담당하며, 인지, 기억, 감정 등 다양한 기능을 조절합니다. 따라서 대뇌의 어느 부분에서 문제가 발생하면 기억 상실이나 기타 인지 기능의 장애가 발생할 수 있습니다. 대뇌는 뇌의 가장 큰 부분으로, 뇌의 표면에 위치한 뇌피질과 그 아래에 있는 심층뇌로 구성되어 있습니다. 이러한 대뇌의 구조와 기능이 원활하게 작동하지 않으면 기억 상실이 발생할 수 있습니다. 따라서 대뇌는 기억 상실과 관련된 가장 중요한 부분으로 간주됩니다.'

In [202]:
output = {"eval_id": 107, "standalone_query": top_agent_result[0], "topk": [], "answer": doc_mapping[top_agent_result[1][0]]['content'], "references": [] }

In [203]:
for doc in top_agent_result[1:4]:
    output['topk'].append(doc[0])
    # doc_mapping[doc[0]]['content']
    output['references'].append({"score" : doc[1], "content" : doc_mapping[doc[0]]['content']})

In [227]:
output

{'eval_id': 78,
 'standalone_query': '나무의 분류 방법 연구',
 'topk': ['c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d',
  '8018337f-15cb-4341-b6fa-e311b4372df9',
  'd42ced41-7d0c-4346-bc0a-11454f5b6121'],
 'answer': '한 학생이 다양한 종류의 나무를 조사하고 있습니다. 이 학생은 성장 속도, 온도 범위, 크기가 비슷한 두 나무를 발견했습니다. 그러나 이 두 나무의 잎과 꽃은 서로 다릅니다. 이러한 특징을 고려하면, 이 나무들은 대체로 같은 속에 속해 있을 것으로 추측됩니다. 같은 속에 속한 나무들은 종류별로 유사한 특징을 가지고 있으며, 이는 생물 분류학에서 중요한 기준 중 하나입니다. 따라서 이 학생의 조사 결과는 나무의 분류와 관련된 중요한 정보를 제공할 수 있습니다. 이러한 조사는 나무의 성장과 생태에 대한 이해를 높이는 데 도움이 될 것입니다.',
 'references': [{'score': 2.703155040740967,
   'content': '한 학생이 다양한 종류의 나무를 조사하고 있습니다. 이 학생은 성장 속도, 온도 범위, 크기가 비슷한 두 나무를 발견했습니다. 그러나 이 두 나무의 잎과 꽃은 서로 다릅니다. 이러한 특징을 고려하면, 이 나무들은 대체로 같은 속에 속해 있을 것으로 추측됩니다. 같은 속에 속한 나무들은 종류별로 유사한 특징을 가지고 있으며, 이는 생물 분류학에서 중요한 기준 중 하나입니다. 따라서 이 학생의 조사 결과는 나무의 분류와 관련된 중요한 정보를 제공할 수 있습니다. 이러한 조사는 나무의 성장과 생태에 대한 이해를 높이는 데 도움이 될 것입니다.'},
  {'score': -3.1997716426849365,
   'content': '나무는 재생 가능한 에너지원으로 간주됩니다. 이는 나무가 석탄이 형성되는 것보다 빠르게 자라기 때문입니다. 나무는 태양 에너지를 흡수하여 광합

In [151]:
x = {"eval_id": 78, "standalone_query": "나무의 분류 방법", "topk": ["c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d", "c18592d1-12b6-4d45-9fcd-efbdd6d7b016", "8018337f-15cb-4341-b6fa-e311b4372df9"], "answer": "나무의 분류를 조사하기 위한 방법은 여러 가지가 있습니다. 먼저, 나무의 잎 모양, 줄기 모양, 꽃과 열매의 유무 등을 관찰하여 나무를 분류할 수 있습니다. 또한, 나무의 특징을 관찰하고 기록하면서 비슷한 특징을 가진 나무끼리 묶어가며 분류할 수 있습니다. 분류는 다양한 방법으로 이루어지며, 관찰과 기록을 통해 나무를 분류하는 것이 중요합니다.", "references": [{"score": 13.007582, "content": "한 학생이 다양한 종류의 나무를 조사하고 있습니다. 이 학생은 성장 속도, 온도 범위, 크기가 비슷한 두 나무를 발견했습니다. 그러나 이 두 나무의 잎과 꽃은 서로 다릅니다. 이러한 특징을 고려하면, 이 나무들은 대체로 같은 속에 속해 있을 것으로 추측됩니다. 같은 속에 속한 나무들은 종류별로 유사한 특징을 가지고 있으며, 이는 생물 분류학에서 중요한 기준 중 하나입니다. 따라서 이 학생의 조사 결과는 나무의 분류와 관련된 중요한 정보를 제공할 수 있습니다. 이러한 조사는 나무의 성장과 생태에 대한 이해를 높이는 데 도움이 될 것입니다."}, {"score": 11.53242, "content": "별은 많은 다른 방법으로 분류할 수 있습니다. 이 중에서 별을 분류하는 데 가장 유용하지 않은 것은 표면 질감입니다. 별은 종종 밤하늘에서의 겉보기 밝기로 분류됩니다. 그러나 표면 질감은 별을 분류하는 데에는 유용하지 않습니다. 표면 질감은 별의 물리적 특성과는 관련이 없으며, 별의 분류에는 다른 요소들이 더 중요합니다. 예를 들어, 별의 크기, 온도, 스펙트럼 등이 별을 분류하는 데에 더 유용한 정보를 제공합니다. 따라서 표면 질감은 별을 분류하는 데에는 가장 유용하지 않은 방법입니다."}, {"score": 11.24757, "content": "나무는 재생 가능한 에너지원으로 간주됩니다. 이는 나무가 석탄이 형성되는 것보다 빠르게 자라기 때문입니다. 나무는 태양 에너지를 흡수하여 광합성을 통해 자라납니다. 이 과정에서 나무는 이산화탄소를 흡수하고 산소를 방출합니다. 이러한 특성으로 인해 나무는 지속적으로 재생되는 에너지원으로 사용될 수 있습니다. 반면, 석탄은 수백만 년 동안 지속적인 압력과 열에 노출되어 형성됩니다. 석탄은 화석 연료로 분류되며, 한 번 사용되면 재생되지 않습니다. 따라서 나무는 석탄보다 더욱 지속 가능한 에너지원으로 간주됩니다."}]}


In [152]:
x

{'eval_id': 78,
 'standalone_query': '나무의 분류 방법',
 'topk': ['c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d',
  'c18592d1-12b6-4d45-9fcd-efbdd6d7b016',
  '8018337f-15cb-4341-b6fa-e311b4372df9'],
 'answer': '나무의 분류를 조사하기 위한 방법은 여러 가지가 있습니다. 먼저, 나무의 잎 모양, 줄기 모양, 꽃과 열매의 유무 등을 관찰하여 나무를 분류할 수 있습니다. 또한, 나무의 특징을 관찰하고 기록하면서 비슷한 특징을 가진 나무끼리 묶어가며 분류할 수 있습니다. 분류는 다양한 방법으로 이루어지며, 관찰과 기록을 통해 나무를 분류하는 것이 중요합니다.',
 'references': [{'score': 13.007582,
   'content': '한 학생이 다양한 종류의 나무를 조사하고 있습니다. 이 학생은 성장 속도, 온도 범위, 크기가 비슷한 두 나무를 발견했습니다. 그러나 이 두 나무의 잎과 꽃은 서로 다릅니다. 이러한 특징을 고려하면, 이 나무들은 대체로 같은 속에 속해 있을 것으로 추측됩니다. 같은 속에 속한 나무들은 종류별로 유사한 특징을 가지고 있으며, 이는 생물 분류학에서 중요한 기준 중 하나입니다. 따라서 이 학생의 조사 결과는 나무의 분류와 관련된 중요한 정보를 제공할 수 있습니다. 이러한 조사는 나무의 성장과 생태에 대한 이해를 높이는 데 도움이 될 것입니다.'},
  {'score': 11.53242,
   'content': '별은 많은 다른 방법으로 분류할 수 있습니다. 이 중에서 별을 분류하는 데 가장 유용하지 않은 것은 표면 질감입니다. 별은 종종 밤하늘에서의 겉보기 밝기로 분류됩니다. 그러나 표면 질감은 별을 분류하는 데에는 유용하지 않습니다. 표면 질감은 별의 물리적 특성과는 관련이 없으며, 별의 분류에는 다른 요소들이 더 중요합니다. 예를 들어, 별의 크기, 온도, 스펙트럼 등이 별을 분

x =

In [150]:
output

{'eval_id': 107,
 'standalone_query': '기억 상실증의 원인',
 'topk': ['25de4ffd-cee4-4f27-907e-fd6b802c6ede',
  'ad56325d-400f-416c-b5de-2053eedecac4',
  'df495f22-6315-42a8-9553-b43ab707b683'],
 'answer': '기억 상실증의 원인',
 'references': ['기억 상실증의 원인']}

In [86]:
top_agent_result

['25de4ffd-cee4-4f27-907e-fd6b802c6ede',
 'ad56325d-400f-416c-b5de-2053eedecac4',
 'df495f22-6315-42a8-9553-b43ab707b683',
 '2566f872-32e5-4def-8ed9-3341740e81e8',
 '0c2d5267-638a-4954-8974-55d28a3fe600']

In [43]:
top_agent_result

'기억 상실증 원인 연구'

In [207]:
with open("/upstage-ai-advanced-ir7/data/eval.jsonl", "r") as f:
    eval_doc_mapping = [json.loads(line) for line in f]
    

In [211]:
for eval_doc in eval_doc_mapping:
    top_agent_result = top_agent(messages)

{'eval_id': 107,
 'msg': [{'role': 'user', 'content': '기억 상실증 걸리면 너무 무섭겠다.'},
  {'role': 'assistant', 'content': '네 맞습니다.'},
  {'role': 'user', 'content': '어떤 원인 때문에 발생하는지 궁금해.'}]}

In [212]:
messages

[{'role': 'user', 'content': '기억 상실증 걸리면 너무 무섭겠다.'},
 {'role': 'assistant', 'content': '네 맞습니다.'},
 {'role': 'user', 'content': '원인이 뭘까.'}]